Выберем веб-сайт с табличными данными, например, сайт с информацией о курсах валют Центрального банка Российской Федерации: https://www.cbr.ru/currency_base/daily/

In [10]:
import csv  # Импортируем модуль csv для работы с файлами CSV
import requests  # Импортируем модуль requests для отправки HTTP-запросов
from lxml import html  # Импортируем модуль html из библиотеки lxml для парсинга HTML
from tabulate import tabulate # Импортируем tabulate для представления читаемой таблицы в консоль

# Установка строки агента пользователя
headers = {  # Создаем словарь headers для хранения заголовков HTTP-запроса
    # Установка строки агента пользователя, чтобы имитировать веб-браузер и избежать блокировки сервером
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Отправка HTTP GET-запроса и получение HTML-содержимого страницы
# URL-адрес страницы с курсами валют
url = 'https://www.cbr.ru/currency_base/daily/'
# Отправляем GET-запрос на указанный URL с заголовками headers
response = requests.get(url, headers=headers)
content = response.content  # Получаем содержимое ответа в виде байтового массива

# Парсинг содержимого HTML с помощью библиотеки lxml
# Создаем объект ElementTree из полученного содержимого
root = html.fromstring(content)

# Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого
# XPath-выражение для выбора таблицы с классом "data"
table_xpath = '//table[@class="data"]'
rows_xpath = './/tr'  # XPath-выражение для выбора строк таблицы
cols_xpath = './/td'  # XPath-выражение для выбора ячеек таблицы

# Получаем первую таблицу, соответствующую XPath-выражению
table = root.xpath(table_xpath)[0]
rows = table.xpath(rows_xpath)  # Получаем все строки таблицы

# Сохранение извлеченных данных в CSV-файл
# Открываем файл currency_rates.csv в режиме записи
with open('currency_rates.csv', 'w', newline='') as csvfile:
    # Определяем имена полей в CSV-файле
    fieldnames = ['Currency', 'Name', 'Rate']
    # Создаем объект DictWriter для записи данных в CSV-файл
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  # Записываем заголовки полей в CSV-файл

    # Перебираем строки таблицы, начиная со второй строки (первая строка - заголовок)
    table_data = []
    for row in rows[1:]:
        cols = row.xpath(cols_xpath)  # Получаем ячейки строки
        if len(cols) >= 3:  # Проверяем, что строка содержит хотя бы 3 ячейки
            # Получаем текстовое содержимое первой ячейки и удаляем лишние пробелы
            currency = int(cols[0].text_content().strip()) / 10
            # Получаем текстовое содержимое второй ячейки и удаляем лишние пробелы
            name = cols[1].text_content().strip()
            # Получаем текстовое содержимое третьей ячейки и удаляем лишние пробелы
            rate = cols[2].text_content().strip()

            # Записываем данные в CSV-файл
            writer.writerow({'Currency': currency, 'Name': name, 'Rate': rate})

            # Вывод таблицы в консоль
            table_data.append([currency, name, rate])

print(tabulate(table_data, headers=[
      'Currency', 'Name', 'Rate'], tablefmt='grid'))

+------------+--------+--------+
|   Currency | Name   |   Rate |
+============+========+========+
|        3.6 | AUD    |      1 |
+------------+--------+--------+
|       94.4 | AZN    |      1 |
+------------+--------+--------+
|        5.1 | AMD    |    100 |
+------------+--------+--------+
|       93.3 | BYN    |      1 |
+------------+--------+--------+
|       97.5 | BGN    |      1 |
+------------+--------+--------+
|       98.6 | BRL    |      1 |
+------------+--------+--------+
|       34.8 | HUF    |    100 |
+------------+--------+--------+
|       41   | KRW    |   1000 |
+------------+--------+--------+
|       70.4 | VND    |  10000 |
+------------+--------+--------+
|       34.4 | HKD    |      1 |
+------------+--------+--------+
|       98.1 | GEL    |      1 |
+------------+--------+--------+
|       20.8 | DKK    |      1 |
+------------+--------+--------+
|       78.4 | AED    |      1 |
+------------+--------+--------+
|       84   | USD    |      1 |
+---------

Этот код сохраняет данные о курсах валют в CSV-файл с именем currency_rates.csv. Каждая строка в файле содержит название валюты, ее название на русском языке и курс к российскому рублю.